In [116]:
def naive_with_counts(pattern, text):
    P, T = len(pattern), len(text)
    occurence = []
    
    character_comparisons = 0 
    alignments_tried = 0
    
    for i in range(T - P + 1):
        alignments_tried += 1
        j = 0 
        while j < P:
            character_comparisons += 1
            if pattern[j] != text[j + i]: 
                break
            j += 1
            
        if j == P: occurence.append(i)
    
    return occurence, alignments_tried, character_comparisons

In [117]:
p = 'word'
t = 'there would have been a time for such a word'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[40] 41 46


In [118]:
p = 'needle'
t = 'needle need noodle needle'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 20 35


In [119]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [120]:
genome = readGenome('019-chr1.GRCh38.excerpt.fasta')

In [123]:
alu = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'

In [124]:
naive_with_counts(alu, genome)

([56922], 799954, 984143)

In [125]:
from bm_preproc import BoyerMoore

In [126]:
def boyer_moore_with_counts(p, p_bm, t):
    i = 0
    occurrences = []
    alignments_tried, character_comparisons = 0, 0
    while i < len(t) - len(p) + 1:
        alignments_tried += 1
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            character_comparisons += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
        
    return occurrences, alignments_tried, character_comparisons

In [127]:
p = 'word'
t = 'there would have been a time for such a word'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[40] 12 15


In [128]:
p = 'needle'
t = 'needle need noodle needle'
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 5 18


In [129]:
p_bm = BoyerMoore(alu)

In [130]:
boyer_moore_with_counts(alu, p_bm, genome)

([56922], 127974, 165191)

In [145]:
import bisect
class Index(object):
    """ Holds a substring index for a text T """

    def __init__(self, t, k):
        """ Create index from all substrings of t of length k """
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer

    def query(self, p):
        n = 2
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            candidate = self.index[i][0]
            misses = 0
            for j in range(0, len(candidate)):
                if candidate[j] != p[j]:
                    misses += 1
                    if misses > n:
                        break
            if misses <= n:   
                hits.append(self.index[i][1])
            i += 1
        return len(hits)

In [146]:
index = Index(genome, 8)
print(index.query('GGCGCGGTGGCTCACGCCTGTAAT'))

300


In [147]:
def naive_2mm(pattern, text):
    P, T = len(pattern), len(text)
    occurence = []
    for i in range(T - P + 1):
        j = 0 
        mm = 0        
        while j < P:
            if pattern[j] != text[j + i]: 
                mm += 1
                if mm > 2: break
            j += 1
            
        if j == P: occurence.append(i)
    
    return occurence

In [184]:
occ = naive_2mm('GGCGCGGTGGCTCACGCCTGTAAT', genome)
print(occ, len(occ))

[56922, 84641, 147558, 160162, 160729, 191452, 262042, 273669, 364263, 421221, 429299, 465647, 551134, 635931, 657496, 681737, 717706, 724927, 747359] 19


In [189]:
print(genome[147558:147558+24])
print('GGCGCGGTGGCTCACGCCTGTAAT')

GGCGCGGTGGCTCATGCCTGTAAT
GGCGCGGTGGCTCACGCCTGTAAT


In [76]:
def app_match(p, t, n):
    segment_len = round(len(t) / (n+1))
    matches = set()
    for i in range(n + 1):
        start = i * segment_len
        end = min((i + 1) * segment_len, len(p))
        bm = BoyerMoore(p[start:end], alphabet='ACGT')
        curr_matches, a, b = boyer_moore_with_counts(p[start:end], bm, t)
        for m in curr_matches:
            if m < start or m-start+len(p) > len(t): continue
                
            misses = 0
            for j in range(0, start):
                if p[j] != p[m-start+j]:
                    misses += 1
                    if misses > n:
                        break
            
            for j in range(end, len(p)):
                if p[j] != p[m-start+j]:
                    misses += 1
                    if misses > n:
                        break                    
            
            if misses <= n:
                matches.add(m-start)
        
        return list(matches), a, b
            
            
                 
        

In [77]:
app_match('GGCGCGGTGGCTCACGCCTGTAAT', genome, 2)

([364263, 717706, 657496, 56922, 262042], 126203, 196873)

In [78]:
naive_with_counts('GGCGCGGTGGCTCACGCCTGTAAT', genome)      

([56922, 262042, 364263, 657496, 717706], 799977, 984116)

In [79]:
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [80]:
ind = SubseqIndex('ATATAT', 3, 2)
print(ind.index)

[('AAA', 0), ('TTT', 1)]


In [81]:
p = 'TTATAT'
print(ind.query(p[0:]))

[]


In [82]:
print(ind.query(p[1:]))


[1]


In [200]:
def query_subseq(p, t, ind):
    n = 2
    all_matches = set()
    num_index_hits = 0
    P = len(p)
    T = len(t)
    for i in range(P - ind.k + 1):
        cut = p[i:]
        matches = ind.query(cut)
#         Extend matching segments to see if whole p matches
        for m in matches:
            num_index_hits += 1
            mismatches = 0
            for j in range(P):
                if not p[j] == t[m+j]:
                    mismatches += 1
                    if mismatches > n:
                        break

            if mismatches <= n:
                all_matches.add(m)
    return list(all_matches), num_index_hits

In [201]:
t = 'to-morrow and to-morrow and to-morrow creeps in this petty pace'
p = 'to-morrow and to-morrow '
subseq_ind = SubseqIndex(t, 8, 3)

In [202]:
occurrences, num_index_hits = query_subseq(p, t, subseq_ind)

In [203]:
print(occurrences)

[0, 14]


In [204]:
print(num_index_hits)

6


In [205]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
subseq_ind = SubseqIndex(genome, 8, 3)

In [206]:
occurrences, num_index_hits = query_subseq(p, genome, subseq_ind)

In [207]:
print(occurrences)

[84641, 273669, 147558, 364263, 681737, 717706, 465647, 747359, 657496, 56922, 635931, 191452, 262042]


In [208]:
print(num_index_hits)

79
